[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/analytics-and-ml/model-training/gpl/03-ce-scoring.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/analytics-and-ml/model-training/gpl/03-ce-scoring.ipynb)

# Cross-Encoder Scoring

The final step in preparing our training data for GPL is a cross encoder scoring step. Given all of the query, passage pairs we've generated, both:

$$Positives = (Q, P^+)$$

<center>and</center>

$$Negatives = (Q, P^-)$$

We pass these into a cross encoder model that is trained to predict similarity scores of *(Q, P)* pairs.

First, we will load our cross encoder model.

In [1]:
from sentence_transformers import CrossEncoder

model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
model

Let's define a generator function to read pairs from file.

In [2]:
from tqdm.auto import tqdm

def get_lines():
    # loop through each file
    with open('data/triplets.tsv', 'r', encoding='utf-8') as fp:
        lines = fp.read().split('\n')
    # loop through each line in the current file
    for line in tqdm(lines):
        q, p, n = line.split('\t')
        # return the query, positive, negative
        yield q, p, n

We use the cross encoder to calculate the similarity of the positive pair, and negative pair, and then take the *score* as the margin between the two similarity scores. We are taking the margin as we will be training our bi-encoder model using Margin MSE loss (which requires the margin/difference).

$$ Margin = sim(Q, P^+) - sim(Q, P^-) $$

In [3]:
lines = get_lines()
label_lines = []

for line in lines:
    q, p, n = line
    p_score = model.predict((q, p))
    n_score = model.predict((q, n))
    margin = p_score - n_score
    # append pairs to label_lines with margin score
    label_lines.append(
        q + '\t' + p + '\t' + n + '\t' + str(margin)
    )

with open("data/triplets_margin.tsv", 'w', encoding='utf-8') as fp:
    fp.write('\n'.join(label_lines))

  0%|          | 0/200000 [00:00<?, ?it/s]

Now we have our *(Q, P<sup>+</sup>, P<sup>-</sup>)* pairs, we can move on to fine-tuning with MarginMSE loss.

---